## Energy-based Flow Classifier (EFC) - Pure Python

### Applying EFC to Bitcoin blockchain in the presence of label scarcity

In [195]:
from math import ceil
from pprint import pprint

In [196]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

In [197]:
from efc_python.classification_functions import (
    one_class_fit,
    one_class_predict,
)
from efc_python.generic_discretize import discretize, get_intervals

In [198]:
from research_aml_elliptic.src.reaml.model_performance import calculate_model_score

In [199]:
from research_aml_elliptic.src.experiments.general_functions.elliptic_data_preprocessing import run_elliptic_preprocessing_pipeline

In [200]:
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    precision_score,
    recall_score,
    roc_auc_score,
)

In [120]:
# Import Elliptic data set and set variables
last_time_step = 49
last_train_time_step = 34
only_labeled = True

In [121]:
# '1': 1, -> class1 (illicit)
# '2': 0, -> class2 (licit)
# 'unknown': 2 -> dropped
X_train, X_test, y_train, y_test = run_elliptic_preprocessing_pipeline(last_train_time_step=last_train_time_step,
                                                                             last_time_step=last_time_step,
                                                                             only_labeled=only_labeled)

/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/research_aml_elliptic/src/experiments/general_functions/elliptic_data_preprocessing.py:46: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_classes.replace({"class": {"1": 1, "2": 0, "unknown": 2}}, inplace=True)


In [122]:
intervals = get_intervals(X_train, 10)  # get discretization intervals from train set

In [123]:
X_train = discretize(X_train, intervals)  # discretize train
X_test = discretize(X_test, intervals)  # discretize test

/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc_python/generic_discretize.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.iloc[:, feature].fillna(len(intervals[feature]), inplace=True)
/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc_python/generic_discretize.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '29894   NaN
29895   NaN
29896   NaN
29897   NaN
29898   NaN
         ..


In [124]:
idx_abnormal = np.where(y_train == 1)[0]  # find abnormal samples indexes in the training set

In [125]:
X_train.drop(idx_abnormal, axis=0, inplace=True)  # remove abnormal samples from training (EFC trains with only benign instances)

In [126]:
y_train.drop(idx_abnormal, axis=0, inplace=True)  # remove the corresponding abonrmal training targets

In [127]:
# EFC's hyperparameters
Q = X_test.values.max()
LAMBDA = 0.5  # pseudocount parameter

In [128]:
coupling, h_i, cutoff, _, _ = one_class_fit(np.array(X_train), Q, LAMBDA)  # train model

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 166/166 [00:01<00:00, 84.41it/s]


In [129]:
y_predicted, energies = one_class_predict(np.array(X_test), coupling, h_i, cutoff, Q)  # test model

In [130]:
# colect results
confusion_matrix = confusion_matrix(y_test, y_predicted)
print("Single-class results")
print('confusion_matrix', confusion_matrix)

Single-class results
confusion_matrix [[10278  5309]
 [  857   226]]


In [131]:
model_score = calculate_model_score(y_true=y_test.values, y_pred=y_predicted, metric="f1")

In [132]:
model_score

np.float64(0.06829857963130856)

In [133]:
y_true = y_test.values
y_pred = y_predicted

In [166]:
metric_dict = {
        "contamination": "100%",
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred, pos_label=1),
        "f1_micro": f1_score(y_true, y_pred, average="micro"),
        "f1_macro": f1_score(y_true, y_pred, average="macro"),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "roc_auc": roc_auc_score(y_true, y_pred),
        "model_score": model_score,
    }

In [167]:
pprint(metric_dict)

{'accuracy': 0.6301139772045591,
 'contamination': '100%',
 'f1': np.float64(0.06829857963130856),
 'f1_macro': np.float64(0.4187761889998471),
 'f1_micro': np.float64(0.6301139772045591),
 'model_score': np.float64(0.06829857963130856),
 'precision': np.float64(0.04083107497741644),
 'recall': np.float64(0.20867959372114497),
 'roc_auc': np.float64(0.4340376219712416)}


In [168]:
columns_metrics = ["contamination", "accuracy", "f1", "f1_micro", "f1_macro", "precision", "recall", "roc_auc", "model_score"]

In [169]:
df_efc_metrics = pd.DataFrame(data=metric_dict, columns=columns_metrics, index=[0])

In [170]:
df_efc_metrics

,contamination,accuracy,f1,f1_micro,f1_macro,precision,recall,roc_auc,model_score
0,100%,0.630114,0.068299,0.630114,0.418776,0.040831,0.20868,0.434038,0.068299


In [180]:
columns = ["contamination", "True Negative", "False positive", "False Negative", "True Positive"]

In [181]:
confusion_matrix = {
            "contamination": "100%",
            "True Negative": 10278,
            "False positive": 5309,
            "False Negative": 857,
            "True Positive": 226
},

In [182]:
df_efc_confusion_matrix = pd.DataFrame(data=confusion_matrix, columns=columns)

In [183]:
df_efc_confusion_matrix

,contamination,True Negative,False positive,False Negative,True Positive
0,100%,10278,5309,857,226


### Train EFC With 5% Labeled Elliptic Data Set

In [184]:
# elliptic data set from reaml repo
X_train_5, X_test_5, y_train_5, y_test_5 = run_elliptic_preprocessing_pipeline(last_train_time_step=last_train_time_step,
                                                                             last_time_step=last_time_step,
                                                                             only_labeled=only_labeled)

/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/research_aml_elliptic/src/experiments/general_functions/elliptic_data_preprocessing.py:46: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_classes.replace({"class": {"1": 1, "2": 0, "unknown": 2}}, inplace=True)


In [185]:
# efc preps
intervals = get_intervals(X_train_5, 10)  # get discretization intervals from train set
X_train_5 = discretize(X_train_5, intervals)  # discretize train
X_test_5 = discretize(X_test_5, intervals)  # discretize test

/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc_python/generic_discretize.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.iloc[:, feature].fillna(len(intervals[feature]), inplace=True)
/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc_python/generic_discretize.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '29894   NaN
29895   NaN
29896   NaN
29897   NaN
29898   NaN
         ..


In [186]:
indices_illicit = np.where(y_train_5 == 1)[0]

In [187]:
drop_indices_illicit = np.random.choice(indices_illicit, size=ceil(len(indices_illicit) * 0.95))

In [188]:
# retrieve idxs abnormals and choose 95% of them
# abnormals == class1 (illicit)
# drop random labeled indices
X_train_5.drop(drop_indices_illicit, axis=0, inplace=True)  # remove abnormal samples from training (EFC trains with only benign instances)
y_train_5.drop(drop_indices_illicit, axis=0, inplace=True)  # remove the corresponding abonrmal training targets

In [189]:
# EFC's hyperparameters
Q = np.int64(X_test.values.max())
LAMBDA = 0.5  # pseudocount parameter

In [190]:
coupling, h_i, cutoff, _, _ = one_class_fit(np.array(X_train_5), Q, LAMBDA)  # train model

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 166/166 [00:01<00:00, 87.88it/s]


In [192]:
y_predicted_5, energies = one_class_predict(np.array(X_test_5), coupling, h_i, cutoff, Q)  # test model

In [201]:
# colect results
confusion_matrix_5 = confusion_matrix(y_test_5, y_predicted_5)
print("Single-class results")
print('confusion_matrix', confusion_matrix_5)

Single-class results
confusion_matrix [[10273  5314]
 [  891   192]]


In [202]:
model_score_5 = calculate_model_score(y_true=y_test.values, y_pred=y_predicted, metric="f1")

In [203]:
y_true_5 = y_test_5.values
y_pred_5 = y_predicted_5

In [204]:
metric_dict_5 = {
        "contamination": "5%",
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred, pos_label=1),
        "f1_micro": f1_score(y_true, y_pred, average="micro"),
        "f1_macro": f1_score(y_true, y_pred, average="macro"),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "roc_auc": roc_auc_score(y_true, y_pred),
        "model_score": model_score_5,
    }

In [205]:
pprint(metric_dict_5)

{'accuracy': 0.6301139772045591,
 'contamination': '5%',
 'f1': np.float64(0.06829857963130856),
 'f1_macro': np.float64(0.4187761889998471),
 'f1_micro': np.float64(0.6301139772045591),
 'model_score': np.float64(0.06829857963130856),
 'precision': np.float64(0.04083107497741644),
 'recall': np.float64(0.20867959372114497),
 'roc_auc': np.float64(0.4340376219712416)}


In [208]:
df_efc_metrics.loc[1] = metric_dict_5

In [209]:
df_efc_metrics

,contamination,accuracy,f1,f1_micro,f1_macro,precision,recall,roc_auc,model_score
0,100%,0.630114,0.068299,0.630114,0.418776,0.040831,0.20868,0.434038,0.068299
1,5%,0.630114,0.068299,0.630114,0.418776,0.040831,0.20868,0.434038,0.068299


In [214]:
confusion_matrix_5 = {
            "contamination": "5%",
            "True Negative": 10273,
            "False positive": 5314,
            "False Negative": 891,
            "True Positive": 192
}

In [215]:
df_efc_confusion_matrix.loc[1] = confusion_matrix_5

In [216]:
df_efc_confusion_matrix

,contamination,True Negative,False positive,False Negative,True Positive
0,100%,10278,5309,857,226
1,5%,10273,5314,891,192


### Train EFC With 10% Labeled Elliptic Data Set

In [ ]:
# elliptic data set from reaml repo
X_train, X_test, y_train, y_test = run_elliptic_preprocessing_pipeline(last_train_time_step=last_train_time_step,
                                                                             last_time_step=last_time_step,
                                                                             only_labeled=only_labeled)

In [ ]:
# efc preps
intervals = get_intervals(X_train, 10)  # get discretization intervals from train set
X_train = discretize(X_train, intervals)  # discretize train
X_test = discretize(X_test, intervals)  # discretize test

In [ ]:
indices_illicit = np.where(y_train == 1)[0]

In [ ]:
drop_indices_illicit = np.random.choice(indices_illicit, size=ceil(len(indices_illicit) * 0.90))

In [ ]:
# retrieve idxs abnormals and choose 90% of them
# abnormals == class1 (illicit)
# drop random labeled indices
X_train.drop(drop_indices_illicit, axis=0, inplace=True)  # remove abnormal samples from training (EFC trains with only benign instances)
y_train.drop(drop_indices_illicit, axis=0, inplace=True)  # remove the corresponding abonrmal training targets

In [ ]:
X_train

In [ ]:
# EFC's hyperparameters
Q = np.int64(X_test.values.max())
LAMBDA = 0.5  # pseudocount parameter

In [ ]:
coupling, h_i, cutoff, _, _ = one_class_fit(np.array(X_train), Q, LAMBDA)  # train model

In [ ]:
y_predicted, energies = one_class_predict(np.array(X_test), coupling, h_i, cutoff, Q)  # test model

In [ ]:
model_score = calculate_model_score(y_true=y_test.values, y_pred=y_predicted, metric="f1")
model_score

In [ ]:
y_true = y_test.values
y_pred = y_predicted

In [ ]:
metric_dict = {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred, pos_label=1),
        "f1_micro": f1_score(y_true, y_pred, average="micro"),
        "f1_macro": f1_score(y_true, y_pred, average="macro"),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "roc_auc": roc_auc_score(y_true, y_pred),
    }

In [ ]:
pprint(metric_dict)

In [ ]:
# colect results
print("Single-class results")
print('confusion_matrix', confusion_matrix(y_test, y_predicted))

### Train EFC With 100% Labeled Elliptic Data Set

In [ ]:
# elliptic data set from reaml repo
X_train, X_test, y_train, y_test = run_elliptic_preprocessing_pipeline(last_train_time_step=last_train_time_step,
                                                                             last_time_step=last_time_step,
                                                                             only_labeled=only_labeled)

In [ ]:
# efc preps
intervals = get_intervals(X_train, 10)  # get discretization intervals from train set
X_train = discretize(X_train, intervals)  # discretize train
X_test = discretize(X_test, intervals)  # discretize test

In [ ]:
# EFC's hyperparameters
Q = np.int64(X_test.values.max())
LAMBDA = 0.5  # pseudocount parameter

In [ ]:
coupling, h_i, cutoff, _, _ = one_class_fit(np.array(X_train), Q, LAMBDA)  # train model

In [ ]:
y_predicted, energies = one_class_predict(np.array(X_test), coupling, h_i, cutoff, Q)  # test model

In [ ]:
model_score = calculate_model_score(y_true=y_test.values, y_pred=y_predicted, metric="f1")
model_score

In [ ]:
y_true = y_test.values
y_pred = y_predicted

In [ ]:
metric_dict = {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred, pos_label=1),
        "f1_micro": f1_score(y_true, y_pred, average="micro"),
        "f1_macro": f1_score(y_true, y_pred, average="macro"),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "roc_auc": roc_auc_score(y_true, y_pred),
    }

In [ ]:
pprint(metric_dict)

### Train EFC Without Labeled Data

In [ ]:
# elliptic data set from reaml repo
X_train, X_test, y_train, y_test = run_elliptic_preprocessing_pipeline(last_train_time_step=last_train_time_step,
                                                                             last_time_step=last_time_step,
                                                                             only_labeled=only_labeled)

In [ ]:
# efc preps
intervals = get_intervals(X_train, 10)  # get discretization intervals from train set
X_train = discretize(X_train, intervals)  # discretize train
X_test = discretize(X_test, intervals)  # discretize test

In [ ]:
idx_abnormal = np.where(y_train == 1)[0]  # find abnormal samples indexes in the training set
X_train.drop(idx_abnormal, axis=0, inplace=True)  # remove abnormal samples from training (EFC trains with only benign instances)
y_train.drop(idx_abnormal, axis=0, inplace=True)  # remove the corresponding abonrmal training targets

In [ ]:
# EFC's hyperparameters
Q = np.int64(X_test.values.max())
LAMBDA = 0.5  # pseudocount parameter

In [ ]:
coupling, h_i, cutoff, _, _ = one_class_fit(np.array(X_train), Q, LAMBDA)  # train model

In [ ]:
y_predicted, energies = one_class_predict(np.array(X_test), coupling, h_i, cutoff, Q)  # test model

In [ ]:
model_score = calculate_model_score(y_true=y_test.values, y_pred=y_predicted, metric="f1")

In [ ]:
y_true = y_test.values
y_pred = y_predicted

In [ ]:
metric_dict = {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred, pos_label=1),
        "f1_micro": f1_score(y_true, y_pred, average="micro"),
        "f1_macro": f1_score(y_true, y_pred, average="macro"),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "roc_auc": roc_auc_score(y_true, y_pred),
    }

In [ ]:
pprint(metric_dict)

### Train EFC Without Discretization

In [ ]:
# elliptic data set from reaml repo
X_train, X_test, y_train, y_test = run_elliptic_preprocessing_pipeline(last_train_time_step=last_train_time_step,
                                                                             last_time_step=last_time_step,
                                                                             only_labeled=only_labeled)

In [ ]:
# EFC's hyperparameters
Q = np.int64(X_test.values.max())
LAMBDA = 0.5  # pseudocount parameter

In [ ]:
# train model
coupling, h_i, cutoff, _, _ = one_class_fit(np.array(X_train), Q, LAMBDA)

In [ ]:
y_predicted, energies = one_class_predict(np.array(X_test), coupling, h_i, cutoff, Q)  # test model

In [ ]:
model_score = calculate_model_score(y_true=y_test.values, y_pred=y_predicted, metric="f1")

In [ ]:
y_true = y_test.values
y_pred = y_predicted

In [ ]:
metric_dict = {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred, pos_label=1),
        "f1_micro": f1_score(y_true, y_pred, average="micro"),
        "f1_macro": f1_score(y_true, y_pred, average="macro"),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "roc_auc": roc_auc_score(y_true, y_pred),
    }

In [ ]:
pprint(metric_dict)